# Orbital Selection Utilties

Utilities for selecting active spaces 

In [5]:
import numpy as np
from pyscf import gto, scf, lo
from pyscf.tools import molden
from pyscf.tools import mo_mapping

/Users/nickrubin/anaconda3/envs/fqe-dev/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [6]:
# files settings
scf_type = 'rhf'
spin = 0
basis = 'ccpvdz'
loc_type = 'noloc'
chkfile_path = 'pm_localized_rohf_ccpvdz_spin5.chk'


In [7]:
# load molecule and scf_data
mol, scf_data = scf.chkfile.load_scf(chkfile_path)
# load split localized MOs
# loc_mo_coeffs = np.load("{}_localized_{}_{}_spin{}_localized_mocoeffs.npy".format(loc_type, scf_type, basis, spin))

In [8]:
# Utility function for looking at coeffs / AO in the MO
def coeff_labeler(mo_coeff_vec, mol, threshold=0.1):
    support_idx = np.where(np.abs(mo_coeff_vec) > threshold)[0]
    for idx in support_idx:
        ao_label = mol.ao_labels()[idx].split(' ')
        ao_label = str(int(ao_label[0]) + 1) + " ".join([xx for xx in ao_label[1:]])
        print(idx+1,  ao_label, mo_coeff_vec[idx])

def ao_label_printer(mol):
    for idx, label in enumerate(mol.ao_labels()):
        print(idx+1, label)
        
def ao_dist(mo_coeffs, mol, ao_list, threshold=0.1):
    cnt = 0
    for ao in ao_list:
        comp = mo_mapping.mo_comps(ao, mol, mo_coeffs)
        print('counter\tMO-id    {} components'.format(ao))
        for i,c in enumerate(comp):
            if np.abs(c) > threshold:
                print('%-3d\t%-3d      %.10f' % (cnt, i, c))
                cnt += 1

def ao_dist_combined(mo_coeffs, mol, ao_list, threshold=0.1):
    cnt = 0
    comp = mo_mapping.mo_comps(ao_list, mol, mo_coeffs)
    print('counter\tMO-id    {} components'.format(" ".join(ao_list)))
    for i,c in enumerate(comp):
        if np.abs(c) > threshold:
            print('%-3d\t%-3d      %.10f' % (cnt, i, c))
            cnt += 1

In [181]:
ao_dist(scf_data['mo_coeff'], mol, ['C 2pz', 'N 2pz'], threshold=0.5)
print()
ao_dist_combined(scf_data['mo_coeff'], mol, ['C 2pz', 'N 2pz'], threshold=0.45)
print()
ao_dist_combined(scf_data['mo_coeff'], mol, ['Fe 3d'], threshold=0.3)
print()
ao_dist_combined(scf_data['mo_coeff'], mol, ['Fe 4d'], threshold=0.3)
print()
ao_dist_combined(scf_data['mo_coeff'], mol, ['S 3p', 'S 3s'], threshold=0.3)

counter	MO-id    C 2pz components
0  	75       0.9767284611
1  	92       0.9735007323
2  	93       0.9910382845
3  	95       0.9733279993
4  	96       0.9711355445
5  	103      0.7830765277
6  	104      0.9099449093
7  	108      0.9643976501
8  	109      0.9728532003
9  	114      0.7453815012
10 	119      0.9712293846
11 	129      0.9683306687
12 	132      0.7517583563
13 	133      0.8023903457
14 	134      0.7923734459
counter	MO-id    N 2pz components
15 	74       0.5817032789
16 	77       0.6641081181
17 	83       0.6611417483
18 	97       0.5848215579

counter	MO-id    C 2pz N 2pz components
0  	47       0.4939865317
1  	74       0.9859476527
2  	75       0.9938209759
3  	77       0.9849787481
4  	79       0.4943949594
5  	83       0.9836408997
6  	92       0.9920000941
7  	93       0.9935671954
8  	95       0.9901104446
9  	96       0.9884294861
10 	97       0.9873916377
11 	98       0.4958513657
12 	99       0.4952272280
13 	103      0.9681514257
14 	104      0.9726477754
15 	106

In [260]:
# porphyrin has a 
aromatic_occ = [(74, 2), (75, 2), (77, 2), (83, 2), (92, 2), (93, 2), (95, 2), (96, 2), 
                (97, 2), (98, 2), (99, 2), (103, 1), (104, 1)]  # 97 is suspect  mostly a  4s-S in 3d Fe
aromatic_virt = [108, 109, 114, 119, 121, 122, 125, 129, 132, 133, 134] # 121 maybe 122 also is kinda the same
print("pi-electrons ", sum([x[1] for x in aromatic_occ]))
print("pi-occ orbs ", len(aromatic_occ))
print("pi-virt orbs ", len(aromatic_virt))
fe3d = [(82, 2), (88, 2), (105, 1), (106, 1), (107, 1)]
fe3d_virt = [137]
cys_occ = [(52, 2), (62, 2), (94, 2), (100, 2), (138, 0) ]
fe4d = [152, 153, 154, 155, 167, 259]
orbital_index = 138
print("Orbital Energy {: 5.85f}".format(scf_data['mo_energy'][orbital_index]))
print("Orbital Occ {: 5f}".format(scf_data['mo_occ'][orbital_index]))
print("Orbital Coefficients")
coeff_labeler(loc_mo_coeffs[:, orbital_index], mol)
print()

pi-electrons  24
pi-occ orbs  13
pi-virt orbs  11
Orbital Energy  0.3454090557835178265655429186153924092650413513183593750000000000000000000000000000000
Orbital Occ  0.000000
Orbital Coefficients
2 1C 2s     0.36046397339277797
3 1C 3s     0.2826702734566321
4 1C 2px    -0.44370956652561955
5 1C 2py    0.46568138985636354
6 1C 2pz    0.19859105761041393
7 1C 3px    -0.2228035689140748
8 1C 3py    0.2340526963418717
9 1C 3pz    0.1363819216513998
31 5S 2s     -0.14549528530684075
32 5S 3s     -0.3877280224432767
33 5S 4s     -0.12839158155732916
37 5S 3px    -0.4119559263476723
38 5S 3py    0.43241194970931524
39 5S 3pz    0.20199341038099053
40 5S 4px    -0.29466586842823556
41 5S 4py    0.30929757965302607
42 5S 4pz    0.15780159618062697
458 42Fe 4pz    0.1382519951916387



In [250]:
# pi system for sto-3g system
# (4 * 5) = 20 Carbons contributing 2pz orbitals
# 4 N2 contributing 2pz orbitals = 6 electrons (4 from pyrroligic and 2 from other N2)
# 24 orbitals with 20 + 2 * 4 = 28 electrons (14 occupied, 10 virt)
# 108 kinda looks like pi*
occ_orbs = aromatic_occ + fe3d + cys_occ[:4]
virt_orbs = aromatic_virt + [cys_occ[-1][0]] + fe4d + fe3d_virt
occ_orbs = [xx[0] for xx in occ_orbs]
occupations = [scf_data['mo_occ'][xx] for xx in occ_orbs + virt_orbs]
print(occupations)

[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [251]:
orb_energies = [scf_data['mo_energy'][xx] for xx in sorted(occ_orbs + virt_orbs)]
for xx in orb_energies:
    print(xx)
print(len(occ_orbs))
print(len(virt_orbs))
print(sum(occupations))
print(occ_orbs)
print(virt_orbs)

-0.9224495690226451
-0.7330135269479728
-0.587492496555132
-0.5721620565067989
-0.5672474995770829
-0.5435428047987265
-0.538390289666352
-0.4950945653521544
-0.4391144477111199
-0.43290961688955104
-0.37981988169155806
-0.37679311198308874
-0.372944405798949
-0.3656195362889749
-0.3546597565406532
-0.3471592777660545
-0.34337591729785655
-0.1956895019087616
-0.17579606569074535
-0.07239094251910093
-0.043418068724443905
-0.02223178397207971
0.025807394745410814
0.09265445288281504
0.1832103616464083
0.20533341801080585
0.21190214199435572
0.2119752251810004
0.23056981775421728
0.266188479795168
0.29411561729352204
0.29778107144260385
0.3016281398166862
0.33995815326835355
0.3454090557835178
0.46211230278945736
0.4644392507973329
0.4686341446706904
0.4796612267023339
0.608825649041476
1.170234337123863
22
19
39.0
[74, 75, 77, 83, 92, 93, 95, 96, 97, 98, 99, 103, 104, 82, 88, 105, 106, 107, 52, 62, 94, 100]
[108, 109, 114, 119, 121, 122, 125, 129, 132, 133, 134, 138, 152, 153, 154, 155,

occ_orbitals = [26, 30, 31, 32, 33]
virt_orbitals = [34, 35, 36, 37, 45, 53, 55, 62, 63, 70]
CASCI Total Energy  -383.3544535258293
CASCI CAS Energy  -13.7156422375474

occ_orbitals = [26, 30, 31, 32, 33]
virt_orbitals = [34, 35, 36, 37, 45]
CASCI Total Energy  -383.33191042526494
CASCI CAS Energy  -13.69309913698305
